In [1]:
import os
import tweepy
import pandas as pd
from pymongo import MongoClient
from sklearn.feature_extraction.text import CountVectorizer

consumer_key =  os.environ['tweety_consumer_key']
consumer_secret = os.environ['tweety_consumer_secret']
access_token = os.environ['tweety_access_token']
acess_token_secret = os.environ['tweety_acess_token_secret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, acess_token_secret)
api = tweepy.API(auth)

In [2]:
conn = MongoClient('localhost', 27017)
database = conn.get_database('twitterdatabase')
col = database.get_collection('tweets')

In [3]:
class StreamListner(tweepy.StreamListener):
    def __init__(self, col):
        super().__init__()
        self.__col__ = col

    def on_status(self, status):
         obj = { 'created_at': status.created_at, 'id_str': status.id_str, 'text': status.text }
         result = self.__col__.insert_one(obj)
         print(obj)

    def on_error(self, status_code):
        if status_code == 420:
            return False

In [4]:
track = ['futebol']

In [5]:
stream_listner = StreamListner(col)
stream = tweepy.Stream(auth=api.auth, listener=stream_listner)
stream.filter(track=track, languages=['pt'], is_async=True)

In [6]:
dataset = [{ "created_at:": item["created_at"], "text": item["text"]  } for item in col.find()]
df = pd.DataFrame(dataset)
df.head(10)

,created_at:,text
0,2020-12-15 21:07:57,RT @Curiosidades_PL: ATENÇÃO: Este perfil não ...
1,2020-12-15 21:07:57,@oporcoiris @porcomunas @globoesporteSP Precis...
2,2020-12-15 21:07:59,Seu natal vai ter panetone? Não porque a mídia...
3,2020-12-15 21:07:59,RT @melaniestan_: Fã de jogador de futebol é t...
4,2020-12-15 21:07:59,Esses são os mesmos que dizem que um é absurdo...
5,2020-12-15 21:08:03,Pegar esse futebol 💪
6,2020-12-15 21:08:04,"n tenho, pelo meu pai acho q flamengo ou sao p..."
7,2020-12-15 21:08:04,Se Lucas Vázquez é jogador de futebol eu sou a...
8,2020-12-15 21:08:04,Futebol vai ser na chuva hoje esqueceeeee
9,2020-12-15 21:08:05,Pedro e Gabigol c uma perna são melhores q os ...


In [7]:
stream.disconnect()

In [8]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df.text)

In [9]:
word_count = pd.DataFrame(cv.get_feature_names(), columns=["word"])
word_count["count"] = count_matrix.sum(axis=0).tolist()[0]
word_count = word_count.sort_values("count", ascending=False).reset_index(drop=True)
word_count[:50]

,word,count
0,de,10
1,futebol,10
2,que,6
3,https,4
4,não,4
5,co,4
6,rt,3
7,mais,3
8,são,3
9,em,3
